### 线性回归的简洁实现
通过实现深度学习框架来简洁地实现线性回归模型生成数据集

In [23]:
import numpy as np
import torch
from torch.utils import data  # 导入包含有关数据加载和处理的工具
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [24]:
# 调用框架中现有的API来读取数据
def load_array(data_arrays,
               batch_size,
               is_train=True):  # 使用该数据迭代器`data_iter`来获取一个批次（batch）的数据
    """构造一个PyTorch数据迭代器"""

    # 创建一个PyTorch的`TensorDataset`对象
    # 并将输入的数据`tensors`（`data_arrays`是一个元组或列表）组合成一个数据集
    # 这里的数据`tensors`包括特征`features`和标签`labels`，每个样本的特征和标签都是一个张量。
    dataset = data.TensorDataset(*data_arrays)  # 将多个tensor组合成dataset

    return data.DataLoader(dataset, batch_size, shuffle=is_train)


batch_size = 10
data_iter = load_array((features, labels), batch_size)

# 使用`iter`函数将数据迭代器转换为迭代器对象，并使用`next`函数获取迭代器中的下一个批次的数据
# 这里`data_iter`是一个数据迭代器，每次调用`next`函数，都会返回一个包含特征和标签的批次数据。
next(iter(data_iter))

[tensor([[ 1.5534,  0.9456],
         [ 0.4744, -0.4386],
         [-0.7837, -0.6368],
         [ 0.2439, -1.2039],
         [-1.0646, -0.3751],
         [-1.0910,  0.4547],
         [ 0.3127,  0.0964],
         [ 0.7525,  0.4823],
         [ 0.2615,  1.1037],
         [ 0.0882, -1.6880]]),
 tensor([[ 4.0784],
         [ 6.6303],
         [ 4.7877],
         [ 8.7744],
         [ 3.3510],
         [ 0.4706],
         [ 4.4962],
         [ 4.0507],
         [ 0.9675],
         [10.1131]])]

`nn.Sequential`类是PyTorch中的一个**容器类，用于构建序列化的神经网络模型。它可以按照顺序组合多个神经网络层，从而构成一个简单的前向神经网络**。

在深度学习中，神经网络通常由多个层组成，例如全连接层、卷积层、池化层等。使用`nn.Sequential`可以将这些层按照顺序组合在一起，形成一个神经网络模型。每个层都是`nn.Module`的子类，因此可以直接添加到`nn.Sequential`中。

创建`nn.Sequential`对象时，**只需要在构造函数中按照顺序传入各个层即可**。例如：

```python
from torch import nn

# 定义一个简单的神经网络模型，包含两个全连接层
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
```

上述代码中，`net`是一个包含两个全连接层的神经网络模型。第一个全连接层的输入维度是784，输出维度是256；然后通过ReLU激活函数进行非线性变换；最后的第二个全连接层的输入维度是256，输出维度是10，用于分类任务的输出。

使用`nn.Sequential`的好处是它简化了神经网络的构建过程，将多个层按照顺序组合在一起，更易于阅读和维护。但**需要注意的是**，`nn.Sequential`适用于顺序式的神经网络，不适用于包含分支或跳跃连接等复杂结构的模型。对于复杂的神经网络，可以通过继承`nn.Module`基类，手动定义前向传播函数，实现更灵活的模型构建。

In [25]:
# 使用框架的预定义好的层
# `nn`是神经网络的缩写
from torch import nn

# `nn.Linear(2, 1)`表示在输入特征维度为2的情况下，使用线性层（全连接层）构建一个输出维度为1的神经网络。
# 即输入特征经过这一层后，输出一个标量结果。
net = nn.Sequential(nn.Linear(2, 1))

In [26]:
# 初始化模型参数

# `net[0].weight.data.normal_(0, 0.01)`: 对神经网络的第一个层（即线性层）的权重进行初始化。
# `net[0]`表示`net`中的第一个层，即线性层。`.weight`表示该层的权重张量，`.data`表示获取权重张量的数据部分。
# `.normal_(0, 0.01)`是一个操作函数，用于将权重张量初始化为来自均值为0，标准差为0.01的正态分布的随机数。
# 这样的初始化有助于在训练过程中更快地收敛。
net[0].weight.data.normal_(0, 0.01)  # type: ignore

# `net[0].bias.data.fill_(0)`: 对神经网络的第一个层（即线性层）的偏置进行初始化。
# `net[0]`表示`net`中的第一个层，即线性层。`.bias`表示该层的偏置张量，`.data`表示获取偏置张量的数据部分。
# `.fill_(0)`是一个操作函数，用于将偏置张量的所有元素初始化为0。
# 这样的初始化可以帮助模型在训练过程中更快地拟合数据。
net[0].bias.data.fill_(0)  # type: ignore

tensor([0.])

In [27]:
# 计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数。默认情况下，它返回所有样本损失的平均值。
loss = nn.MSELoss()

In [28]:
# 实例化`SGD`实例
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [29]:
# 训练过程代码与从零开始实现时所做的非常相似
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad() # 梯度清零
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000250
epoch 2, loss 0.000100
epoch 3, loss 0.000099
